# MIS: Complementaria 13 (2023-20)

En la clase de hoy veremos implementaciones de modelos de asignación. Así, trataremos los modelos de asignación en mercados de matrimonios y aplicaciones a universidades desarrollados por Gale y Shapley. La idea general de la clase es implementar el modelo de aceptación diferida para el mercado de matrimonios (1:1) y quedará como práctica para ustedes realizar las modificaciones pertinentes para implementar este modelo para el caso de aplicaciones a universidades (m:1). 

### Previos

En esta sección se establecen algunas funciones que permitirán desarrollar la propuesta de los algoritmos de la clase.

In [ ]:
# Cargue de paquetes
import numpy as np
import random

In [ ]:
def participants(n_prop,n_disp):
    list_prop = ["P_{}".format(i) for i in range(n_prop)]
    list_disp = ["D_{}".format(i) for i in range(n_disp)]
    return list_prop,list_disp

In [ ]:
def randomizer(list_agents,highlighted_agent):
    n = len(list_agents) # Numero de agentes
    i = random.randint(0,n-1) # Primera randomización para introducir agente especial
    hl_agent_intro = list_agents[:i]+[highlighted_agent]+list_agents[i:] # Introducción del agente especial
    pos = 0 # Inicialización de las posiciones
    n += 1 # Total de agentes incluido agente especial
    dictionary = {} # Inicialización del diccionario de preferencias:
    type_agent = "P" in highlighted_agent
    while n>0:
        i = random.randint(0,n-1)
        agent_selected = hl_agent_intro.pop(i)
        if agent_selected == highlighted_agent:
            relevant_position = pos
        if type_agent:
            dictionary[pos] = agent_selected
        else:
            dictionary[agent_selected] = pos
        pos +=1
        n-=1
    return dictionary,relevant_position        


In [ ]:
def preferences(list_prop,list_disp):
    pref_prop = {i:randomizer(list_disp,i) for i in list_prop}
    pref_disp = {i:randomizer(list_prop,i) for i in list_disp}
    return pref_prop,pref_disp

In [ ]:
def initial_assignation(pref_prop,pref_disp):
    return {i:pref_prop[i][1] for i in pref_prop},{i:i for i in pref_disp}

### Mercado de Matrimonios

A continuación se desarrolla la implementación del algoritmo de aceptación diferida para el Mercado de Matrimonios (1:1). La tarea que tienen es pensar cómo parar el algoritmo de mejor manera (si esta existe).

In [ ]:
def deferred_acceptance_1_1(preferences):
    pref_prop = preferences[0] # Se alistan las preferencias
    pref_disp = preferences[1]
    assignation = initial_assignation(pref_prop,pref_disp) # Se inicializan las asignaciones para cada grupo
    tracker_prop = {i:0 for i in pref_prop}
    able_to_prop = {i:True for i in pref_prop}
    not_complete = True 
    cont_reps=0
    max_reps=300
    while not_complete:
        for prop in tracker_prop:
            print(prop)
            ranking_disp = tracker_prop[prop] # Obtengo el ranking de quien me gusta
            disp = pref_prop[prop][0][ranking_disp] # Obtengo a quien me gusta
            if disp != prop: # Verifico si no se obtiene una elección definitiva conmigo mismo.
                ranking_prop = pref_disp[disp][0][prop] # Miro que tanto le gusto a quien me gusta
                actual = assignation[1][disp] # Obtengo la asignación actual de quien me gusta
                ranking_actual = pref_disp[disp][0][actual] #Obtengo el ranking de la asignación actual para quien me gusta
                if ranking_actual>ranking_prop:# Verifico si le gusto más que su asignación actual a quien me gusta
                    print("Aceptado")
                    assignation[1][disp] = prop # Soy asignado a quien me gusta
                    assignation[0][prop] = ranking_disp # Me asignan a quien me gusta
                    able_to_prop[prop] = False # Como fui asignado a quien me gusta, no propongo
                    if "P" in actual: # Si el ex de quien me gusta era el/ella mismo, entonces no dejamos a nadie. DLC:
                        assignation[0][actual] = pref_prop[actual][1] # A quien dejaron se asigna a si mismo de nuevo
                        able_to_prop[actual] = True # A quien dejaron se le deja proponer de nuevo 
                else: # Si me rechazan, voy a mi siguiente opción
                    print("Rechazado")
                    tracker_prop[prop] = ranking_disp + 1
                print(assignation)
            else: #Ya no puedo elegir a nadie, entonces salgo del mercado
                print("Prop se queda solo")
                able_to_prop[prop] = False
        if all([j==False for j in able_to_prop.values()]): # Condición de parado
            print("all")
            not_complete=False

        if cont_reps>max_reps: # Condición de parado de emergencia (por si algo. Este parámetro puede cambiarse)
            not_complete=False
        cont_reps+=1
    A = {i:pref_prop[i][0][assignation[0][i]] for i in assignation[0]} # Paso las elecciones a un formato amigable
    return A,assignation[1] # Retorno 

    

In [ ]:
PA=participants(3,3)
P=preferences(PA[0],PA[1])

In [ ]:
P

In [ ]:
deferred_acceptance_1_1(P)

### Aplicaciones a Universidades

En las celdas dispuestas abajo, realice modificaciones al código propuesto arriba para implementar el algoritmo de aceptación diferida para el caso más general de las Aplicaciones a Universidades (m:1). 